In [1]:
import torch
import pprint
from context_citer import run_experiment
from context_citer_data_utils import read_data, preprocess_hotpot, preprocess_tydiqa
from subtract import run_and_evaluate_method as subtract_and_evaluate_method
from transformers import AutoModelForCausalLM, AutoTokenizer
from subtract import compute_best_subtraction
from datasets import load_dataset, load_from_disk
from experiment_utils import relaxed_f1
import pandas as pd
import string 
import re
%load_ext autoreload
import evaluate
%autoreload 2


ModuleNotFoundError: No module named 'context_citer'

In [2]:
hotpot_dataset = read_data(data_path = '../data/', name = 'distractor', dataset_name = 'hotpot_qa', split = 'validation')
ex_prompt = preprocess_hotpot(hotpot_dataset, only_yes_no = True)

#preprocess_hotpot(hotpot_dataset, only_yes_no = True)
tydiqa_dataset = read_data(data_path = '../data/', dataset_name = 'tydiqa', name = 'primary_task', split = 'validation')
tydiqa_prompt = preprocess_tydiqa(tydiqa_dataset, only_yes_no = True)

In [3]:
drop_benchmark = read_data(data_path = '../data/', dataset_name="ucinlp/drop", split = 'validation')
drop_benchmark

Dataset({
    features: ['section_id', 'query_id', 'passage', 'question', 'answers_spans'],
    num_rows: 9535
})

In [4]:
torch.cuda.empty_cache()
model, tokenizer = AutoModelForCausalLM.from_pretrained('meta-llama/Llama-3.2-1B-Instruct', device_map = 'auto',load_in_8bit = True,attn_implementation = 'flash_attention_2'), AutoTokenizer.from_pretrained('meta-llama/Llama-3.2-1B-Instruct')


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


In [10]:
row_id =  8088
context = drop_benchmark[row_id]['passage']
print(context)
question = f'{drop_benchmark[row_id]["question"]}. Provide shortest answer possible, long answers are penalized heavily. No explaination is required.\n'
prompt = f'context: {context}\nquestion: {question}'
input_ids = tokenizer(prompt, return_tensors='pt',truncation = True).to(model.device)
model.eval()
with torch.no_grad():
    outputs = model.generate(**input_ids, max_new_tokens=16,do_sample = False)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
print(drop_benchmark[row_id]['answers_spans'])
torch.cuda.empty_cache()


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


, the gender distribution of the population was 50.2% male and 49.8% female.  The age distribution, , in Cunter is; 18 people or 9.1% of the population are between 0 and 9 years old.  7 people or 3.5% are 10 to 14, and 16 people or 8.1% are 15 to 19.  Of the adult population, 29 people or 14.6% of the population are between 20 and 29 years old.  28 people or 14.1% are 30 to 39, 36 people or 18.2% are 40 to 49, and 31 people or 15.7% are 50 to 59.  The senior population distribution is 13 people or 6.6% of the population are between 60 and 69 years old, 15 people or 7.6% are 70 to 79, there are 4 people or 2.0% who are 80 to 89, and there are 1 people or 0.5% who are 90 to 99.
context:, the gender distribution of the population was 50.2% male and 49.8% female.  The age distribution,, in Cunter is; 18 people or 9.1% of the population are between 0 and 9 years old.  7 people or 3.5% are 10 to 14, and 16 people or 8.1% are 15 to 19.  Of the adult population, 29 people or 14.6% of the popul

In [ ]:
relaxed_f1('The RDAF squadron in 1962 comprised Alouette helicopters.','Alouette helicopters')

In [9]:
drop_benchmark[row_id]

{'section_id': 'nfl_1184',
 'query_id': '7dfd2b64-f39e-4bb4-aeb0-1900adda6018',
 'passage': " Hoping to rebound from their loss to the Patriots, the Raiders stayed at home for a Week 16 duel with the Houston Texans.  Oakland would get the early lead in the first quarter as quarterback JaMarcus Russell completed a 20-yard touchdown pass to rookie wide receiver Chaz Schilens.  The Texans would respond with fullback Vonta Leach getting a 1-yard touchdown run, yet the Raiders would answer with kicker Sebastian Janikowski getting a 33-yard and a 30-yard field goal.  Houston would tie the game in the second quarter with kicker Kris Brown getting a 53-yard and a 24-yard field goal. Oakland would take the lead in the third quarter with wide receiver Johnnie Lee Higgins catching a 29-yard touchdown pass from Russell, followed up by an 80-yard punt return for a touchdown.  The Texans tried to rally in the fourth quarter as Brown nailed a 40-yard field goal, yet the Raiders' defense would shut do

In [33]:

theoretically_correct_answer = [{'id': drop_benchmark[row_id]['query_id'], 'answers': {'text':drop_benchmark[row_id]['answers_spans']['spans']}}]
model_answer = {'id': drop_benchmark[row_id]['query_id'], 'prediction_text': 'Both teams kicked 2 field goals in the first half'}
metric = evaluate.load('exact_match')
metric.compute(references = ['Alouette helicopters'], predictions = ['Alouette helicopters'], ignore_case = True, ignore_punctuation = True)


{'exact_match': 1.0}

In [31]:
exact_match = evaluate.load("exact_match")
refs = ["the cat", "theater", "YELLING", "agent007"]
preds = ["cat?", "theater", "yelling", "agent"]
results = exact_match.compute(references=refs, predictions=preds, regexes_to_ignore=["the ", "yell"], ignore_case=True, ignore_punctuation=True)
print(round(results["exact_match"], 2))

0.5


In [12]:
drop_benchmark[3]['answers_spans']

{'spans': ['2', '2', '2', '2', '2'],
 'types': ['number', 'number', 'number', 'number', 'number']}

In [57]:
drop_benchmark[4567]['answer_spans']

{'section_id': 'nfl_336',
 'query_id': '88fc0951-1a7f-406f-ac93-14914a926f39',
 'passage': " The Chargers at home, donned their throwback jerseys and played a Week 9 fight against the Cleveland Browns.  In the first quarter, both sides would limit each other to a field goal.  Chargers kicker Nate Kaeding would complete a 29-yard field goal, while Browns kicker Phil Dawson completed a 37-yard field goal.  In the second quarter, Dawson would help Cleveland take the lead on a 20-yard field goal.  San Diego a defensive touchdown as back as LB Marques Harris recovered a fumble in the Browns end zone.  The Brown's would retake the lead with Dawson nailing a 42-yard and a 30-yard field goal.  In the third quarter, the Chargers RB LaDainian Tomlinson got a 41-yard TD run for the only score of the period. In the fourth quarter, Dawson helped the Browns by converting a 36-yard field goal,  San Diego continued to add to their total with Tomlinson getting a 7-yard TD run.  Dawson would get one mor

In [35]:
natural_questions = read_data(dataset_name="google-research-datasets/natural_questions", name = "default")
natural_questions


KeyboardInterrupt: 

In [27]:
tydiqa_dataset_eng = tydiqa_dataset.filter(lambda x: x['language'].startswith('eng'))
print(tydiqa_dataset_eng[0]['passage_answer_candidates']['plaintext_start_byte'])
print(tydiqa_dataset_eng[0]['passage_answer_candidates']['plaintext_end_byte'])


[1, 357, 1024, 1069, 1417, 1947, 2245, 2719, 3440, 4124, 4595, 6072, 7332, 8187, 8878, 9227, 10731, 11192, 12399, 14042, 15461, 17121, 17598, 17893, 18451, 19822, 20597, 21752, 22933, 24120, 24843, 25722, 27153, 27500]
[333, 1023, 1068, 1416, 1946, 2214, 2718, 3439, 4112, 4565, 6071, 7331, 8186, 8877, 9205, 10730, 11191, 12397, 14040, 15460, 17120, 17597, 17881, 18445, 19821, 20596, 21751, 22911, 24119, 24842, 25698, 27142, 27482, 27709]


In [20]:
print(tydiqa_dataset_eng[0]['document_plaintext'])


The zebra finch (Taeniopygia guttata)[2] is the most common estrildid finch of Central Australia and ranges over most of the continent, avoiding only the cool moist south and some areas of the tropical far north. It can also be found natively in Indonesia and East Timor. The bird has been introduced to Puerto Rico and Portugal.[1]
Taxonomy and phylogeny
The zebra finch was first collected in 1801 during Nicolas Baudin's expedition to Australia. It was described in 1817 by Louis Jean Pierre Vieillot in his Nouveau Dictionnaire d'Histoire Naturelle, where he gave it the scientific name Fringilla guttata. The Australian subspecies was then described in 1837 by John Gould as Amadina castanotis.[3] Its current genus, Taeniopygia, was described in 1862 by Ludwig Reichenbach.[4] It is placed in the tribe Poephilini,[5] along with the genus Poephila, which it was previously included in;[6] the split between Taeniopygia and Poephila is justified by 1987 study using protein electrophoresis and 

In [70]:
tydiqa_dataset.filter(lambda x: x['id'].startswith('english'))['context']

["Wound care encourages and speeds wound healing via cleaning and protection from reinjury or infection. Depending on each patient's needs, it can range from the simplest first aid to entire nursing specialties such as wound, ostomy, and continence nursing and burn center care.",
 'Brothers Amos and Wilfrid Ayre founded Burntisland Shipbuilding Co. in 1918 as a First World War emergency shipyard.[1] Its yard at Burntisland West Dock had four berths and capacity to build ships up to 450 feet (140m) long[1] and up to 59 feet (18m) beam.[3] However, until the 1950s Burntisland built relatively few vessels more than about 425 feet (130m) long and 57 feet (17.4m) beam.[3]',
 'The cerebral cortex is folded in a way that allows a large surface area of neural tissue to fit within the confines of the neurocranium. When unfolded, each hemispheric cortex has a total surface area of about 1.3 square feet (0.12m2).[5] The folding is inward away from the surface of the brain, and is also present on 

In [47]:
tydiqa_prompt[0][1]

{'passage_answer_candidates': {'plaintext_start_byte': [6,
   971,
   1918,
   2689,
   2879,
   3602,
   4192,
   5112,
   5471,
   6194,
   6577,
   7080,
   8011,
   9631,
   10799,
   11720,
   12709,
   13096,
   13621,
   14181,
   14870,
   15270,
   15545,
   15962,
   16469,
   16711,
   16955,
   17328,
   17597,
   18314,
   18602,
   19030,
   19601,
   20161,
   20860,
   21751,
   22222,
   22589,
   23199,
   23680,
   24341,
   24725,
   25206,
   25736,
   26307,
   26822,
   26924,
   27463],
  'plaintext_end_byte': [970,
   1899,
   2677,
   2841,
   3601,
   4191,
   5110,
   5433,
   6193,
   6576,
   7069,
   8009,
   9630,
   10797,
   11719,
   12707,
   13095,
   13620,
   14171,
   14850,
   15269,
   15544,
   15961,
   16468,
   16690,
   16927,
   17308,
   17573,
   18313,
   18587,
   19009,
   19600,
   20135,
   20859,
   21726,
   22221,
   22588,
   23198,
   23679,
   24340,
   24724,
   25205,
   25735,
   26280,
   26812,
   26899,
   27460,
   277

In [58]:
from transformers import AutoModelForCausalLM, AutoTokenizer
model, tokenizer = AutoModelForCausalLM.from_pretrained('meta-llama/Llama-3.2-1B-Instruct', device_map = 'auto',load_in_8bit = True,attn_implementation = 'flash_attention_2'), AutoTokenizer.from_pretrained('meta-llama/Llama-3.2-1B-Instruct')


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


In [63]:
#tokenizer.padding_side = 'left'
tokenizer.pad_token = tokenizer.eos_token
prompts = ['What is the capital of India? Please provide the shortest answer possible, long answers are penalized heavily. No explaination is required.', 'Capital of Canada? Please provide the shortest answer possible, long answers are penalized heavily. No explaination is required.']
input_ids = tokenizer(prompts, return_tensors='pt',padding = True).to(model.device)
model.eval()
with torch.no_grad():
    outputs = model.generate(input_ids['input_ids'], attention_mask = input_ids['attention_mask'], max_new_tokens=16,num_return_sequences = 1, do_sample = False, output_scores=True, return_dict_in_generate=True)

/home/abhi/.local/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/abhi/.local/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [64]:
tokenizer.batch_decode(outputs['sequences'], skip_special_tokens=False)

['<|begin_of_text|>What is the capital of India? Please provide the shortest answer possible, long answers are penalized heavily. No explaination is required. The answer is: New Delhi.<|eot_id|>',
 '<|eot_id|><|eot_id|><|eot_id|><|begin_of_text|>Capital of Canada? Please provide the shortest answer possible, long answers are penalized heavily. No explaination is required. Ottawa.<|eot_id|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|>']

In [75]:
import numpy as np
transition_scores = model.compute_transition_scores(outputs['sequences'], outputs['scores'], normalize_logits=True) # batch_size, generated_tokens
input_length = input_ids['input_ids'].shape[1]   # batch size, seq len
generated_tokens = outputs['sequences'][:, input_length:] # batch_size, generated_tokens and includes end of text token
sequence_probs = []
transition_scores = transition_scores.detach().cpu().numpy()
generated_tokens = generated_tokens.detach().cpu().numpy()
for i in range(len(generated_tokens)):
    sequence_prob = 0.0
    for tok, score in zip(generated_tokens[i], transition_scores[i]):
        if tok == tokenizer.pad_token_id:
            break
        else:
                sequence_prob += score
    sequence_prob = np.exp(sequence_prob)
    sequence_prob = np.log(sequence_prob/(1.0 - sequence_prob))
    sequence_probs.append(np.exp(sequence_prob))


sequence_probs

# for i in range(len(generated_tokens)):
#         sequence_prob = 1.0
#         for tok, score in zip(generated_tokens[i], transition_scores[i]):
#                 token_prob = np.exp(score.detach().cpu().numpy())
#                 sequence_prob += np.exp(score.detach().cpu().numpy())
#                 print(f"| {tok:5d} | {tokenizer.decode(tok):8s} | {score.detach().cpu().numpy():.4f} | {np.exp(score.detach().cpu().numpy()):.2%}")

        #print(tokenizer.decode(generated_tokens[i]))


#print(generated_tokens.shape)

#print(input_length.shape
# # generated_tokens = outputs['sequences'][:, input_length:]

# for tok, score in zip(generated_tokens[0], transition_scores[0]):
#         print(f"| {tok:5d} | {tokenizer.decode(tok):8s} | {score.detach().cpu().numpy():.4f} | {np.exp(score.detach().cpu().numpy()):.2%}")


# for tok, score in zip(generated_tokens[1], transition_scores[1]):
#         print(f"| {tok:5d} | {tokenizer.decode(tok):8s} | {score.detach().cpu().numpy():.4f} | {np.exp(score.detach().cpu().numpy()):.2%}")

[0.004635703643405753, 0.021388065809939908]

In [67]:
eot_id = tokenizer.convert_tokens_to_ids('<|eot_id|>')
end_of_text_id = tokenizer.convert_tokens_to_ids('<|end_of_text|>')
eot_id, end_of_text_id

#outputs['scores'][0].shape


(128009, 128001)

In [22]:
outputs['sequences'].shape


torch.Size([2, 24])

['What is the capital of India? Please provide the shortest answer possible, long answers are penalized heavily. No explaination is required. The answer is: New Delhi.',
 'What is the capital of France? Please provide the shortest answer possible, long answers are penalized heavily. No explaination is required. Paris is the capital of France.']